# Baseline vs Models - Drug Relapse Dataset - Boostrap Test Set

In [1]:
import pandas as pd
import yaml
from importlib import resources as impresources
from recurrent_health_events_prediction import configs
import plotly.express as px
import os
from recurrent_health_events_prediction.model_analysis.utils import MetricEnum

In [2]:
with open((impresources.files(configs) / 'model_config.yaml')) as f:
    model_config = yaml.safe_load(f)
with open((impresources.files(configs) / 'data_config.yaml')) as f:
    data_config = yaml.safe_load(f)

In [3]:
SAVE_RESULTS_DIR = data_config['final_results']["relapse_path"]
filename = "hmm_vs_baseline"

In [14]:
DATASET = "relapse"
BASELINE_RESULTS_DIR = "/workspaces/master-thesis-recurrent-health-events-prediction/_models/drug_relapse/classifiers_baselines"
BASE_MODEL_DIR = model_config[DATASET]["hidden_markov"]["save_model_path"]
CLASSIFIER = "lgbm"
METRIC = MetricEnum.AUC.value
N_BOOTSTRAPS = 5000
RANDOM_STATE = 42
THRESHOLD = 0.5 # probability threshold for classification

In [15]:
filename = f"hmm_vs_baseline_{DATASET}_{CLASSIFIER}_{METRIC}.csv"

## hmm_binary_relapse_30_days

In [30]:
from recurrent_health_events_prediction.model_analysis.utils import get_model_config


model_name = "hmm_binary_relapse_30_days"

get_model_config(model_name, BASE_MODEL_DIR)

{'continuous_event_time_col': 'LOG_TIME_UNTIL_NEXT_POSITIVE',
 'event_id_col': 'COLLECTION_ID',
 'event_name': 'relapse',
 'event_time_cat_col': 'RELAPSE_DURATION_CATEGORY',
 'event_time_cat_order': ['0-30', '30-90', '90-180', '180+'],
 'event_time_feature_col': 'RELAPSE_30_DAYS',
 'feature_define_state_labels': 'RELAPSE_30_DAYS',
 'features': {'LOG_TIME_SINCE_LAST_POSITIVE': 'gaussian',
  'NUM_POSITIVES_SINCE_LAST_NEGATIVE': 'poisson',
  'RELAPSE_30_DAYS': 'bernoulli'},
 'hidden_state_labels': ['low', 'medium_1', 'medium_2', 'high'],
 'id_col': 'DONOR_ID',
 'model_name': 'hmm_binary_relapse_30_days',
 'n_states': 4,
 'save_model_path': '/workspaces/master-thesis-recurrent-health-events-prediction/_models/drug_relapse/hmm',
 'time_col': 'RELAPSE_START',
 'use_only_sequences_gte_2_steps': False}

In [31]:
from recurrent_health_events_prediction.model_analysis.utils import compare_classifier_with_baseline

results1_df = compare_classifier_with_baseline(
    model_name=model_name,
    base_model_dir=BASE_MODEL_DIR,
    baseline_results_dir=BASELINE_RESULTS_DIR,
    classifier=CLASSIFIER,
    metric=METRIC,
    n_boot=N_BOOTSTRAPS,
    random_state=RANDOM_STATE,
    threshold = THRESHOLD,
)

Comparing hmm_binary_relapse_30_days with baseline using auc metric...
Number of predictions in baseline: 2458
Number of predictions in hmm_binary_relapse_30_days: 2458
Number of common predictions: 2458
Evaluating HMM Features with classifier:  lgbm
Calculating  auc  with  5000  bootstrap samples...



  0%|          | 0/5000 [00:00<?, ?it/s]

100%|██████████| 5000/5000 [00:15<00:00, 320.57it/s]


In [32]:
results1_df

,model_name,n_states,classifier,metric,obs_delta,delta_mean,ci_low,ci_high,p_value,n_boot
0,hmm_binary_relapse_30_days,4,lgbm,auc,-0.00538,-0.005247,-0.015043,0.004627,0.2948,5000


## hmm_categorical_relapse_time

In [33]:
from recurrent_health_events_prediction.model_analysis.utils import get_model_config


model_name = "hmm_categorical_relapse_time"

get_model_config(model_name, BASE_MODEL_DIR)

{'add_distribution_params': {'RELAPSE_DURATION_CATEGORY_ENCODED': {'labels': ['0-30',
    '30-90',
    '90-180',
    '180+'],
   'n_categories': 4}},
 'continuous_event_time_col': 'LOG_TIME_UNTIL_NEXT_POSITIVE',
 'event_id_col': 'COLLECTION_ID',
 'event_name': 'relapse',
 'event_time_cat_col': 'RELAPSE_DURATION_CATEGORY',
 'event_time_cat_order': ['0-30', '30-90', '90-180', '180+'],
 'event_time_feature_col': 'RELAPSE_DURATION_CATEGORY_ENCODED',
 'feature_define_state_labels': 'RELAPSE_DURATION_CATEGORY_ENCODED',
 'features': {'LOG_TIME_SINCE_LAST_POSITIVE': 'gaussian',
  'NUM_POSITIVES_SINCE_LAST_NEGATIVE': 'poisson',
  'RELAPSE_DURATION_CATEGORY_ENCODED': 'categorical'},
 'hidden_state_labels': ['high', 'medium', 'low'],
 'id_col': 'DONOR_ID',
 'model_name': 'hmm_categorical_relapse_time',
 'n_states': 3,
 'save_model_path': '/workspaces/master-thesis-recurrent-health-events-prediction/_models/drug_relapse/hmm',
 'time_col': 'RELAPSE_START',
 'use_only_sequences_gte_2_steps': False}

In [34]:
from recurrent_health_events_prediction.model_analysis.utils import compare_classifier_with_baseline

results6_df = compare_classifier_with_baseline(
    model_name=model_name,
    base_model_dir=BASE_MODEL_DIR,
    baseline_results_dir=BASELINE_RESULTS_DIR,
    classifier=CLASSIFIER,
    metric=METRIC,
    n_boot=N_BOOTSTRAPS,
    random_state=RANDOM_STATE,
    threshold = THRESHOLD,
)

Comparing hmm_categorical_relapse_time with baseline using auc metric...
Number of predictions in baseline: 2458
Number of predictions in hmm_categorical_relapse_time: 2458
Number of common predictions: 2458
Evaluating HMM Features with classifier:  lgbm
Calculating  auc  with  5000  bootstrap samples...



  0%|          | 0/5000 [00:00<?, ?it/s]

100%|██████████| 5000/5000 [00:18<00:00, 267.20it/s]


In [35]:
results6_df

,model_name,n_states,classifier,metric,obs_delta,delta_mean,ci_low,ci_high,p_value,n_boot
0,hmm_categorical_relapse_time,3,lgbm,auc,-0.009995,-0.00993,-0.020264,0.000315,0.0584,5000


## hmm_log_normal_relapse_time_drug_vars

In [27]:
from recurrent_health_events_prediction.model_analysis.utils import get_model_config


model_name = "hmm_log_normal_relapse_time_drug_vars"

get_model_config(model_name, BASE_MODEL_DIR)

{'continuous_event_time_col': 'LOG_TIME_UNTIL_NEXT_POSITIVE',
 'event_id_col': 'COLLECTION_ID',
 'event_name': 'relapse',
 'event_time_cat_col': 'RELAPSE_DURATION_CATEGORY',
 'event_time_cat_order': ['0-30', '30-90', '90-180', '180+'],
 'event_time_feature_col': 'LOG_TIME_UNTIL_NEXT_POSITIVE',
 'feature_define_state_labels': 'LOG_TIME_UNTIL_NEXT_POSITIVE',
 'features': {'ALCOHOL_POS': 'bernoulli',
  'BUPRENORPHINE_POS': 'bernoulli',
  'LOG_TIME_SINCE_LAST_NEGATIVE': 'gaussian',
  'LOG_TIME_SINCE_LAST_POSITIVE': 'gaussian',
  'LOG_TIME_UNTIL_NEXT_POSITIVE': 'gaussian',
  'NUM_POSITIVES_SINCE_LAST_NEGATIVE': 'poisson'},
 'hidden_state_labels': ['high', 'medium', 'low'],
 'id_col': 'DONOR_ID',
 'model_name': 'hmm_log_normal_relapse_time_drug_vars',
 'n_states': 3,
 'save_model_path': '/workspaces/master-thesis-recurrent-health-events-prediction/_models/drug_relapse/hmm',
 'time_col': 'RELAPSE_START',
 'use_only_sequences_gte_2_steps': False}

In [28]:
from recurrent_health_events_prediction.model_analysis.utils import compare_classifier_with_baseline

results2_df = compare_classifier_with_baseline(
    model_name=model_name,
    base_model_dir=BASE_MODEL_DIR,
    baseline_results_dir=BASELINE_RESULTS_DIR,
    classifier=CLASSIFIER,
    metric=METRIC,
    n_boot=N_BOOTSTRAPS,
    random_state=RANDOM_STATE,
    threshold = THRESHOLD,
)

Comparing hmm_log_normal_relapse_time_drug_vars with baseline using auc metric...
Number of predictions in baseline: 2458
Number of predictions in hmm_log_normal_relapse_time_drug_vars: 2458
Number of common predictions: 2458
Evaluating HMM Features with classifier:  lgbm
Calculating  auc  with  5000  bootstrap samples...



100%|██████████| 5000/5000 [00:17<00:00, 282.08it/s]


In [29]:
results2_df

,model_name,n_states,classifier,metric,obs_delta,delta_mean,ci_low,ci_high,p_value,n_boot
0,hmm_log_normal_relapse_time_drug_vars,3,lgbm,auc,-0.005028,-0.004925,-0.014614,0.004877,0.3184,5000


## hmm_binary_relapse_30_days_drug_vars

In [37]:
from recurrent_health_events_prediction.model_analysis.utils import get_model_config


model_name = "hmm_binary_relapse_30_days_drug_vars"

get_model_config(model_name, BASE_MODEL_DIR)

{'continuous_event_time_col': 'LOG_TIME_UNTIL_NEXT_POSITIVE',
 'event_id_col': 'COLLECTION_ID',
 'event_name': 'relapse',
 'event_time_cat_col': 'RELAPSE_DURATION_CATEGORY',
 'event_time_cat_order': ['0-30', '30-90', '90-180', '180+'],
 'event_time_feature_col': 'RELAPSE_30_DAYS',
 'feature_define_state_labels': 'RELAPSE_30_DAYS',
 'features': {'ALCOHOL_POS': 'bernoulli',
  'BUPRENORPHINE_POS': 'bernoulli',
  'LOG_TIME_SINCE_LAST_NEGATIVE': 'gaussian',
  'LOG_TIME_SINCE_LAST_POSITIVE': 'gaussian',
  'NUM_POSITIVES_SINCE_LAST_NEGATIVE': 'poisson',
  'RELAPSE_30_DAYS': 'bernoulli'},
 'hidden_state_labels': ['low', 'medium', 'high'],
 'id_col': 'DONOR_ID',
 'model_name': 'hmm_binary_relapse_30_days_drug_vars',
 'n_states': 3,
 'save_model_path': '/workspaces/master-thesis-recurrent-health-events-prediction/_models/drug_relapse/hmm',
 'time_col': 'RELAPSE_START',
 'use_only_sequences_gte_2_steps': False}

In [38]:
from recurrent_health_events_prediction.model_analysis.utils import compare_classifier_with_baseline

results3_df = compare_classifier_with_baseline(
    model_name=model_name,
    base_model_dir=BASE_MODEL_DIR,
    baseline_results_dir=BASELINE_RESULTS_DIR,
    classifier=CLASSIFIER,
    metric=METRIC,
    n_boot=N_BOOTSTRAPS,
    random_state=RANDOM_STATE,
    threshold = THRESHOLD,
)

Comparing hmm_binary_relapse_30_days_drug_vars with baseline using auc metric...
Number of predictions in baseline: 2458
Number of predictions in hmm_binary_relapse_30_days_drug_vars: 2458
Number of common predictions: 2458
Evaluating HMM Features with classifier:  lgbm
Calculating  auc  with  5000  bootstrap samples...



100%|██████████| 5000/5000 [00:31<00:00, 160.24it/s]


In [39]:
results3_df

,model_name,n_states,classifier,metric,obs_delta,delta_mean,ci_low,ci_high,p_value,n_boot
0,hmm_binary_relapse_30_days_drug_vars,3,lgbm,auc,-0.00804,-0.007901,-0.017451,0.001526,0.1064,5000


## hmm_gamma_relapse_time

In [24]:
from recurrent_health_events_prediction.model_analysis.utils import get_model_config


model_name = "hmm_gamma_relapse_time"

get_model_config(model_name, BASE_MODEL_DIR)

{'continuous_event_time_col': 'LOG_TIME_UNTIL_NEXT_POSITIVE',
 'event_id_col': 'COLLECTION_ID',
 'event_name': 'relapse',
 'event_time_cat_col': 'RELAPSE_DURATION_CATEGORY',
 'event_time_cat_order': ['0-30', '30-90', '90-180', '180+'],
 'event_time_feature_col': 'LOG_TIME_UNTIL_NEXT_POSITIVE',
 'feature_define_state_labels': 'LOG_TIME_UNTIL_NEXT_POSITIVE',
 'features': {'LOG_TIME_SINCE_LAST_NEGATIVE': 'gaussian',
  'LOG_TIME_UNTIL_NEXT_POSITIVE': 'gamma',
  'NUM_POSITIVES_SINCE_LAST_NEGATIVE': 'poisson'},
 'hidden_state_labels': ['high_2', 'high_1', 'medium', 'low_2', 'low_1'],
 'id_col': 'DONOR_ID',
 'model_name': 'hmm_gamma_relapse_time',
 'n_states': 5,
 'save_model_path': '/workspaces/master-thesis-recurrent-health-events-prediction/_models/drug_relapse/hmm',
 'time_col': 'RELAPSE_START',
 'use_only_sequences_gte_2_steps': False}

In [25]:
from recurrent_health_events_prediction.model_analysis.utils import compare_classifier_with_baseline

results4_df = compare_classifier_with_baseline(
    model_name=model_name,
    base_model_dir=BASE_MODEL_DIR,
    baseline_results_dir=BASELINE_RESULTS_DIR,
    classifier=CLASSIFIER,
    metric=METRIC,
    n_boot=N_BOOTSTRAPS,
    random_state=RANDOM_STATE,
    threshold = THRESHOLD,
)

Comparing hmm_gamma_relapse_time with baseline using auc metric...
Number of predictions in baseline: 2458
Number of predictions in hmm_gamma_relapse_time: 2458
Number of common predictions: 2458
Evaluating HMM Features with classifier:  lgbm
Calculating  auc  with  5000  bootstrap samples...



  0%|          | 0/5000 [00:00<?, ?it/s]

100%|██████████| 5000/5000 [00:14<00:00, 337.81it/s]


In [26]:
results4_df

,model_name,n_states,classifier,metric,obs_delta,delta_mean,ci_low,ci_high,p_value,n_boot
0,hmm_gamma_relapse_time,5,lgbm,auc,-0.009275,-0.009136,-0.019445,0.000792,0.0708,5000


## hmm_student_t_relapse_time

In [21]:
from recurrent_health_events_prediction.model_analysis.utils import get_model_config


model_name = "hmm_student_t_relapse_time"

get_model_config(model_name, BASE_MODEL_DIR)

{'continuous_event_time_col': 'LOG_TIME_UNTIL_NEXT_POSITIVE',
 'dof': 10,
 'event_id_col': 'COLLECTION_ID',
 'event_name': 'relapse',
 'event_time_cat_col': 'RELAPSE_DURATION_CATEGORY',
 'event_time_cat_order': ['0-30', '30-90', '90-180', '180+'],
 'event_time_feature_col': 'LOG_TIME_UNTIL_NEXT_POSITIVE',
 'feature_define_state_labels': 'LOG_TIME_UNTIL_NEXT_POSITIVE',
 'features': {'LOG_TIME_SINCE_LAST_POSITIVE': 'student_t',
  'LOG_TIME_UNTIL_NEXT_POSITIVE': 'student_t',
  'NUM_POSITIVES_SINCE_LAST_NEGATIVE': 'poisson'},
 'hidden_state_labels': ['high_2', 'high_1', 'medium', 'low_2', 'low_1'],
 'id_col': 'DONOR_ID',
 'model_name': 'hmm_student_t_relapse_time',
 'n_states': 5,
 'save_model_path': '/workspaces/master-thesis-recurrent-health-events-prediction/_models/drug_relapse/hmm',
 'time_col': 'RELAPSE_START',
 'use_only_sequences_gte_2_steps': False}

In [22]:
from recurrent_health_events_prediction.model_analysis.utils import compare_classifier_with_baseline

results5_df = compare_classifier_with_baseline(
    model_name=model_name,
    base_model_dir=BASE_MODEL_DIR,
    baseline_results_dir=BASELINE_RESULTS_DIR,
    classifier=CLASSIFIER,
    metric=METRIC,
    n_boot=N_BOOTSTRAPS,
    random_state=RANDOM_STATE,
    threshold = THRESHOLD,
)

Comparing hmm_student_t_relapse_time with baseline using auc metric...
Number of predictions in baseline: 2458
Number of predictions in hmm_student_t_relapse_time: 2458
Number of common predictions: 2458
Evaluating HMM Features with classifier:  lgbm
Calculating  auc  with  5000  bootstrap samples...



  0%|          | 0/5000 [00:00<?, ?it/s]

100%|██████████| 5000/5000 [00:14<00:00, 338.00it/s]


In [23]:
results5_df

,model_name,n_states,classifier,metric,obs_delta,delta_mean,ci_low,ci_high,p_value,n_boot
0,hmm_student_t_relapse_time,5,lgbm,auc,-0.005412,-0.005365,-0.01519,0.004424,0.2772,5000


## hmm_log_normal_relapse_time

In [18]:
from recurrent_health_events_prediction.model_analysis.utils import get_model_config


model_name = "hmm_log_normal_relapse_time"

get_model_config(model_name, BASE_MODEL_DIR)

{'continuous_event_time_col': 'LOG_TIME_UNTIL_NEXT_POSITIVE',
 'event_id_col': 'COLLECTION_ID',
 'event_name': 'relapse',
 'event_time_cat_col': 'RELAPSE_DURATION_CATEGORY',
 'event_time_cat_order': ['0-30', '30-90', '90-180', '180+'],
 'event_time_feature_col': 'LOG_TIME_UNTIL_NEXT_POSITIVE',
 'feature_define_state_labels': 'LOG_TIME_UNTIL_NEXT_POSITIVE',
 'features': {'LOG_TIME_SINCE_LAST_POSITIVE': 'gaussian',
  'LOG_TIME_UNTIL_NEXT_POSITIVE': 'gaussian',
  'NUM_POSITIVES_SINCE_LAST_NEGATIVE': 'poisson'},
 'hidden_state_labels': ['high', 'medium_2', 'medium_1', 'low'],
 'id_col': 'DONOR_ID',
 'model_name': 'hmm_log_normal_relapse_time',
 'n_states': 4,
 'save_model_path': '/workspaces/master-thesis-recurrent-health-events-prediction/_models/drug_relapse/hmm',
 'time_col': 'RELAPSE_START',
 'use_only_sequences_gte_2_steps': False}

In [19]:
from recurrent_health_events_prediction.model_analysis.utils import compare_classifier_with_baseline

results8_df = compare_classifier_with_baseline(
    model_name=model_name,
    base_model_dir=BASE_MODEL_DIR,
    baseline_results_dir=BASELINE_RESULTS_DIR,
    classifier=CLASSIFIER,
    metric=METRIC,
    n_boot=N_BOOTSTRAPS,
    random_state=RANDOM_STATE,
    threshold = THRESHOLD,
)

Comparing hmm_log_normal_relapse_time with baseline using auc metric...
Number of predictions in baseline: 2458
Number of predictions in hmm_log_normal_relapse_time: 2458
Number of common predictions: 2458
Evaluating HMM Features with classifier:  lgbm
Calculating  auc  with  5000  bootstrap samples...



100%|██████████| 5000/5000 [00:16<00:00, 296.52it/s]


In [20]:
results8_df

,model_name,n_states,classifier,metric,obs_delta,delta_mean,ci_low,ci_high,p_value,n_boot
0,hmm_log_normal_relapse_time,4,lgbm,auc,-0.006788,-0.006716,-0.015878,0.002367,0.1444,5000


## Reduced Features Set - hmm_log_normal_relapse_time

In [16]:
from recurrent_health_events_prediction.model_analysis.utils import get_model_config


model_name = "hmm_relapse_reduced_feats_set"

get_model_config(model_name, BASE_MODEL_DIR)

{'continuous_event_time_col': 'LOG_TIME_UNTIL_NEXT_POSITIVE',
 'event_id_col': 'COLLECTION_ID',
 'event_name': 'relapse',
 'event_time_cat_col': 'RELAPSE_DURATION_CATEGORY',
 'event_time_cat_order': ['0-30', '30-90', '90-180', '180+'],
 'event_time_feature_col': 'LOG_TIME_UNTIL_NEXT_POSITIVE',
 'feature_define_state_labels': 'LOG_TIME_UNTIL_NEXT_POSITIVE',
 'features': {'ALCOHOL_POS': 'bernoulli',
  'BUPRENORPHINE_POS': 'bernoulli',
  'LOG_TIME_SINCE_LAST_NEGATIVE': 'gaussian',
  'LOG_TIME_SINCE_LAST_POSITIVE': 'gaussian',
  'LOG_TIME_UNTIL_NEXT_POSITIVE': 'gaussian',
  'NUM_POSITIVES_SINCE_LAST_NEGATIVE': 'poisson'},
 'hidden_state_labels': ['high', 'medium', 'low'],
 'id_col': 'DONOR_ID',
 'model_name': 'hmm_relapse_reduced_feats_set',
 'n_states': 3,
 'save_model_path': '/workspaces/master-thesis-recurrent-health-events-prediction/_models/drug_relapse/hmm',
 'time_col': 'RELAPSE_START',
 'use_only_sequences_gte_2_steps': False}

In [17]:
from recurrent_health_events_prediction.model_analysis.utils import compare_classifier_with_baseline

results7_df = compare_classifier_with_baseline(
    model_name=model_name,
    base_model_dir=BASE_MODEL_DIR,
    baseline_results_dir=BASELINE_RESULTS_DIR,
    classifier=CLASSIFIER,
    metric=METRIC,
    n_boot=N_BOOTSTRAPS,
    random_state=RANDOM_STATE,
    threshold = THRESHOLD,
)

Comparing hmm_relapse_reduced_feats_set with baseline using auc metric...
Number of predictions in baseline: 2458
Number of predictions in hmm_relapse_reduced_feats_set: 2458
Number of common predictions: 2458
Evaluating HMM Features with classifier:  lgbm
Calculating  auc  with  5000  bootstrap samples...



100%|██████████| 5000/5000 [00:12<00:00, 386.92it/s]


In [13]:
results7_df

,model_name,n_states,classifier,metric,obs_delta,delta_mean,ci_low,ci_high,p_value,n_boot
0,hmm_relapse_reduced_feats_set,3,lgbm,auc,-0.013585,-0.013703,-0.024357,-0.002747,0.0192,2500


## All Results

In [40]:
all_results_df_list = [results1_df, results2_df, results3_df, results4_df, results5_df, results6_df, results7_df, results8_df]
all_results_df = pd.concat(all_results_df_list, ignore_index=True).sort_values(by="delta_mean")


In [41]:
all_results_df

,model_name,n_states,classifier,metric,obs_delta,delta_mean,ci_low,ci_high,p_value,n_boot
6,hmm_relapse_reduced_feats_set,3,lgbm,auc,-0.013585,-0.013551,-0.024399,-0.002544,0.0192,5000
5,hmm_categorical_relapse_time,3,lgbm,auc,-0.009995,-0.009930,-0.020264,0.000315,0.0584,5000
3,hmm_gamma_relapse_time,5,lgbm,auc,-0.009275,-0.009136,-0.019445,0.000792,0.0708,5000
2,hmm_binary_relapse_30_days_drug_vars,3,lgbm,auc,-0.008040,-0.007901,-0.017451,0.001526,0.1064,5000
7,hmm_log_normal_relapse_time,4,lgbm,auc,-0.006788,-0.006716,-0.015878,0.002367,0.1444,5000
4,hmm_student_t_relapse_time,5,lgbm,auc,-0.005412,-0.005365,-0.015190,0.004424,0.2772,5000
0,hmm_binary_relapse_30_days,4,lgbm,auc,-0.005380,-0.005247,-0.015043,0.004627,0.2948,5000
1,hmm_log_normal_relapse_time_drug_vars,3,lgbm,auc,-0.005028,-0.004925,-0.014614,0.004877,0.3184,5000


In [42]:
if not os.path.exists(SAVE_RESULTS_DIR):
    os.makedirs(SAVE_RESULTS_DIR, exist_ok=True)
all_results_df.to_csv(os.path.join(SAVE_RESULTS_DIR, filename), index=False)